In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('../../')

In [5]:
import os 
path = '/Volumes/Ayush_Drive/mnist/'
embedding_path = 'embeddings/cifar100/'
if os.path.exists(path):
    prefix = path
else:
    prefix = ''

In [6]:
import numpy as np
import pandas as pd
import torch 
# the analysis function given the dataset (dont even need the model tbh, pull the activations from elsewhere)
def find_max_indices(activations, topk=10): 
    # columnwise top 10 
    top_k = min(topk, activations.shape[0])
    activations_t = activations.T
    top_values, top_indices = torch.topk(activations_t, top_k, dim=1)

    # convert into dataframe 
    values_np = top_values.cpu().numpy()
    indices_np = top_indices.cpu().numpy()

    neuron_ids = np.arange(activations.shape[1])

    df = pd.DataFrame({
        'neuron_id': neuron_ids,
        'top_values': list(values_np),
        'top_indices': list(indices_np)
    })

    return df

In [6]:
for depth in range(1, 11):
    acts = torch.load(os.path.join(path, 'laion', f'sae_depth_{depth}_decoder_acts.pt'), map_location='cpu')
    
    if depth > 1: 
        acts = acts.T

    df = find_max_indices(activations=acts.detach(), topk=10)
    df.to_parquet(os.path.join(path, 'laion', 'analysis', f'sae_depth_{depth}_decoder_data.parquet'), index=False)
    print(len(df))

65536
65536
16384
8192
4096
2304
2304
2304
2304
2304


In [7]:
import torch 
import pandas as pd 
import os 

df_map = {}
for depth in range(1, 11):
    df = pd.read_parquet(os.path.join(path, 'laion', 'analysis', f'sae_depth_{depth}_decoder_data.parquet'))
    df_map[depth] = df

In [10]:
df_map[2]

,neuron_id,top_values,top_indices
0,0,"[0.010952795, 0.010489352, 0.0070078857, 0.006...","[744, 998, 723, 455, 252, 949, 457, 636, 276, ..."
1,1,"[0.010952795, 0.010489352, 0.0070078857, 0.006...","[744, 998, 723, 455, 252, 949, 457, 636, 276, ..."
2,2,"[0.010952795, 0.010489352, 0.0070078857, 0.006...","[744, 998, 723, 455, 252, 949, 457, 636, 276, ..."
3,3,"[0.010833718, 0.010613878, 0.0070440797, 0.006...","[744, 998, 723, 455, 252, 949, 457, 276, 636, 72]"
4,4,"[0.010904016, 0.010615651, 0.0069562015, 0.006...","[744, 998, 723, 455, 252, 949, 276, 457, 636, 72]"
...,...,...,...
65531,65531,"[0.010952795, 0.010489352, 0.0070078857, 0.006...","[744, 998, 723, 455, 252, 949, 457, 636, 276, ..."
65532,65532,"[0.010952795, 0.010489352, 0.0070078857, 0.006...","[744, 998, 723, 455, 252, 949, 457, 636, 276, ..."
65533,65533,"[0.010952795, 0.010489352, 0.0070078857, 0.006...","[744, 998, 723, 455, 252, 949, 457, 636, 276, ..."
65534,65534,"[0.010952795, 0.010489352, 0.0070078857, 0.006...","[744, 998, 723, 455, 252, 949, 457, 636, 276, ..."


In [11]:
df_map[3]

,neuron_id,top_values,top_indices
0,0,"[0.007221947, 0.0061645256, 0.0061542755, 0.00...","[636, 702, 227, 788, 441, 336, 889, 225, 952, ..."
1,1,"[0.007221947, 0.0061645256, 0.0061542755, 0.00...","[636, 702, 227, 788, 441, 336, 889, 225, 952, ..."
2,2,"[0.007221947, 0.0061645256, 0.0061542755, 0.00...","[636, 702, 227, 788, 441, 336, 889, 225, 952, ..."
3,3,"[0.007221947, 0.0061645256, 0.0061542755, 0.00...","[636, 702, 227, 788, 441, 336, 889, 225, 952, ..."
4,4,"[0.007221947, 0.0061645256, 0.0061542755, 0.00...","[636, 702, 227, 788, 441, 336, 889, 225, 952, ..."
...,...,...,...
16379,16379,"[0.007221947, 0.0061645256, 0.0061542755, 0.00...","[636, 702, 227, 788, 441, 336, 889, 225, 952, ..."
16380,16380,"[0.007221947, 0.0061645256, 0.0061542755, 0.00...","[636, 702, 227, 788, 441, 336, 889, 225, 952, ..."
16381,16381,"[0.007221947, 0.0061645256, 0.0061542755, 0.00...","[636, 702, 227, 788, 441, 336, 889, 225, 952, ..."
16382,16382,"[0.007221947, 0.0061645256, 0.0061542755, 0.00...","[636, 702, 227, 788, 441, 336, 889, 225, 952, ..."


In [12]:
df_map[4]

,neuron_id,top_values,top_indices
0,0,"[0.0026650445, 0.0024426193, 0.0022142134, 0.0...","[294, 686, 334, 733, 370, 547, 480, 51, 995, 119]"
1,1,"[0.0026650445, 0.0024426193, 0.0022142134, 0.0...","[294, 686, 334, 733, 370, 547, 480, 51, 995, 119]"
2,2,"[0.0026650445, 0.0024426193, 0.0022142134, 0.0...","[294, 686, 334, 733, 370, 547, 480, 51, 995, 119]"
3,3,"[0.0026650445, 0.0024426193, 0.0022142134, 0.0...","[294, 686, 334, 733, 370, 547, 480, 51, 995, 119]"
4,4,"[0.0026650445, 0.0024426193, 0.0022142134, 0.0...","[294, 686, 334, 733, 370, 547, 480, 51, 995, 119]"
...,...,...,...
8187,8187,"[0.0026650445, 0.0024426193, 0.0022142134, 0.0...","[294, 686, 334, 733, 370, 547, 480, 51, 995, 119]"
8188,8188,"[0.0026650445, 0.0024426193, 0.0022142134, 0.0...","[294, 686, 334, 733, 370, 547, 480, 51, 995, 119]"
8189,8189,"[0.0026650445, 0.0024426193, 0.0022142134, 0.0...","[294, 686, 334, 733, 370, 547, 480, 51, 995, 119]"
8190,8190,"[0.0026650445, 0.0024426193, 0.0022142134, 0.0...","[294, 686, 334, 733, 370, 547, 480, 51, 995, 119]"


In [13]:
df_map[5]

,neuron_id,top_values,top_indices
0,0,"[0.0025405153, 0.0024839514, 0.00205555, 0.001...","[753, 101, 365, 71, 546, 540, 133, 314, 122, 112]"
1,1,"[0.0025405153, 0.0024839514, 0.00205555, 0.001...","[753, 101, 365, 71, 546, 540, 133, 314, 122, 112]"
2,2,"[0.0025405153, 0.0024839514, 0.00205555, 0.001...","[753, 101, 365, 71, 546, 540, 133, 314, 122, 112]"
3,3,"[0.0025405153, 0.0024839514, 0.00205555, 0.001...","[753, 101, 365, 71, 546, 540, 133, 314, 122, 112]"
4,4,"[0.0025405153, 0.0024839514, 0.00205555, 0.001...","[753, 101, 365, 71, 546, 540, 133, 314, 122, 112]"
...,...,...,...
4091,4091,"[0.0025405153, 0.0024839514, 0.00205555, 0.001...","[753, 101, 365, 71, 546, 540, 133, 314, 122, 112]"
4092,4092,"[0.0025405153, 0.0024839514, 0.00205555, 0.001...","[753, 101, 365, 71, 546, 540, 133, 314, 122, 112]"
4093,4093,"[0.0025357087, 0.0024806277, 0.002056066, 0.00...","[753, 101, 365, 71, 546, 540, 133, 314, 122, 112]"
4094,4094,"[0.0025405153, 0.0024839514, 0.00205555, 0.001...","[753, 101, 365, 71, 546, 540, 133, 314, 122, 112]"


In [14]:
df_map[6]

,neuron_id,top_values,top_indices
0,0,"[0.004694876, 0.004461522, 0.0044036643, 0.004...","[146, 822, 606, 684, 552, 564, 10, 1001, 918, ..."
1,1,"[0.004694876, 0.004461522, 0.0044036643, 0.004...","[146, 822, 606, 684, 552, 564, 10, 1001, 918, ..."
2,2,"[0.004694876, 0.004461522, 0.0044036643, 0.004...","[146, 822, 606, 684, 552, 564, 10, 1001, 918, ..."
3,3,"[0.004694876, 0.004461522, 0.0044036643, 0.004...","[146, 822, 606, 684, 552, 564, 10, 1001, 918, ..."
4,4,"[0.004694876, 0.004461522, 0.0044036643, 0.004...","[146, 822, 606, 684, 552, 564, 10, 1001, 918, ..."
...,...,...,...
2299,2299,"[0.004694876, 0.004461522, 0.0044036643, 0.004...","[146, 822, 606, 684, 552, 564, 10, 1001, 918, ..."
2300,2300,"[0.004694876, 0.004461522, 0.0044036643, 0.004...","[146, 822, 606, 684, 552, 564, 10, 1001, 918, ..."
2301,2301,"[0.004694876, 0.004461522, 0.0044036643, 0.004...","[146, 822, 606, 684, 552, 564, 10, 1001, 918, ..."
2302,2302,"[0.004694876, 0.004461522, 0.0044036643, 0.004...","[146, 822, 606, 684, 552, 564, 10, 1001, 918, ..."


In [15]:
df_map[7]

,neuron_id,top_values,top_indices
0,0,"[0.0034280117, 0.0033865161, 0.003331426, 0.00...","[822, 606, 146, 10, 684, 552, 1001, 918, 493, ..."
1,1,"[0.0034280117, 0.0033865161, 0.003331426, 0.00...","[822, 606, 146, 10, 684, 552, 1001, 918, 493, ..."
2,2,"[0.0034280117, 0.0033865161, 0.003331426, 0.00...","[822, 606, 146, 10, 684, 552, 1001, 918, 493, ..."
3,3,"[0.0034280117, 0.0033865161, 0.003331426, 0.00...","[822, 606, 146, 10, 684, 552, 1001, 918, 493, ..."
4,4,"[0.0034280117, 0.0033865161, 0.003331426, 0.00...","[822, 606, 146, 10, 684, 552, 1001, 918, 493, ..."
...,...,...,...
2299,2299,"[0.0034280117, 0.0033865161, 0.003331426, 0.00...","[822, 606, 146, 10, 684, 552, 1001, 918, 493, ..."
2300,2300,"[0.0034280117, 0.0033865161, 0.003331426, 0.00...","[822, 606, 146, 10, 684, 552, 1001, 918, 493, ..."
2301,2301,"[0.0034280117, 0.0033865161, 0.003331426, 0.00...","[822, 606, 146, 10, 684, 552, 1001, 918, 493, ..."
2302,2302,"[0.0034280117, 0.0033865161, 0.003331426, 0.00...","[822, 606, 146, 10, 684, 552, 1001, 918, 493, ..."


In [17]:
df_map[8]

,neuron_id,top_values,top_indices
0,0,"[0.004470442, 0.00441045, 0.0043235673, 0.0042...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
1,1,"[0.004470442, 0.00441045, 0.0043235673, 0.0042...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
2,2,"[0.004470442, 0.00441045, 0.0043235673, 0.0042...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
3,3,"[0.004470442, 0.00441045, 0.0043235673, 0.0042...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
4,4,"[0.004470442, 0.00441045, 0.0043235673, 0.0042...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
...,...,...,...
2299,2299,"[0.004470442, 0.00441045, 0.0043235673, 0.0042...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
2300,2300,"[0.004470442, 0.00441045, 0.0043235673, 0.0042...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
2301,2301,"[0.004470442, 0.00441045, 0.0043235673, 0.0042...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
2302,2302,"[0.004470442, 0.00441045, 0.0043235673, 0.0042...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."


In [18]:
df_map[9]

,neuron_id,top_values,top_indices
0,0,"[0.00363276, 0.0035511642, 0.0034641551, 0.003...","[606, 146, 822, 10, 684, 1001, 552, 918, 995, ..."
1,1,"[0.00363276, 0.0035511642, 0.0034641551, 0.003...","[606, 146, 822, 10, 684, 1001, 552, 918, 995, ..."
2,2,"[0.00363276, 0.0035511642, 0.0034641551, 0.003...","[606, 146, 822, 10, 684, 1001, 552, 918, 995, ..."
3,3,"[0.00363276, 0.0035511642, 0.0034641551, 0.003...","[606, 146, 822, 10, 684, 1001, 552, 918, 995, ..."
4,4,"[0.00363276, 0.0035511642, 0.0034641551, 0.003...","[606, 146, 822, 10, 684, 1001, 552, 918, 995, ..."
...,...,...,...
2299,2299,"[0.00363276, 0.0035511642, 0.0034641551, 0.003...","[606, 146, 822, 10, 684, 1001, 552, 918, 995, ..."
2300,2300,"[0.00363276, 0.0035511642, 0.0034641551, 0.003...","[606, 146, 822, 10, 684, 1001, 552, 918, 995, ..."
2301,2301,"[0.00363276, 0.0035511642, 0.0034641551, 0.003...","[606, 146, 822, 10, 684, 1001, 552, 918, 995, ..."
2302,2302,"[0.00363276, 0.0035511642, 0.0034641551, 0.003...","[606, 146, 822, 10, 684, 1001, 552, 918, 995, ..."


In [19]:
df_map[10]

,neuron_id,top_values,top_indices
0,0,"[0.004411869, 0.004330666, 0.0042630527, 0.004...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
1,1,"[0.004411869, 0.004330666, 0.0042630527, 0.004...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
2,2,"[0.004411869, 0.004330666, 0.0042630527, 0.004...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
3,3,"[0.004411869, 0.004330666, 0.0042630527, 0.004...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
4,4,"[0.004411869, 0.004330666, 0.0042630527, 0.004...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
...,...,...,...
2299,2299,"[0.004411869, 0.004330666, 0.0042630527, 0.004...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
2300,2300,"[0.004411869, 0.004330666, 0.0042630527, 0.004...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
2301,2301,"[0.004411869, 0.004330666, 0.0042630527, 0.004...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."
2302,2302,"[0.004411869, 0.004330666, 0.0042630527, 0.004...","[146, 606, 822, 684, 552, 10, 1001, 918, 633, ..."


In [21]:
import pandas as pd
import numpy as np
from collections import Counter

# Function to count index patterns
def count_index_patterns(df):
    # Convert string representations of lists to tuples
    if isinstance(df['top_indices'].iloc[0], str):
        # If your data is stored as strings, convert them to actual lists first
        patterns = df['top_indices'].apply(lambda x: tuple(eval(x)))
    else:
        # If already in list format
        patterns = df['top_indices'].apply(tuple)
    
    # Count occurrences
    pattern_counts = Counter(patterns)
    
    # Get the most common patterns
    return pattern_counts

def filter_df(index):
    # Apply the function to count patterns
    pattern_counts = count_index_patterns(df_map[index])

    print("Most common top_indices patterns:")
    for pattern, count in pattern_counts.most_common(10):
        print(f"Pattern occurs {count} times: {pattern[:5]}... (truncated)")

    # Get the most common pattern (the one with the highest count)
    most_common_pattern = pattern_counts.most_common(1)[0][0]

    # Now filter out all rows that have this most common pattern
    if isinstance(df_map[index]['top_indices'].iloc[0], str):
        # If stored as strings
        filtered_df = df_map[index][df_map[index]['top_indices'].apply(lambda x: tuple(eval(x))) != most_common_pattern]
    else:
        # If stored as lists
        filtered_df = df_map[index][df_map[index]['top_indices'].apply(tuple) != most_common_pattern]
    
    return filtered_df

In [22]:
for depth in range(1, 11): 
    filtered_df = filter_df(depth)
    filtered_df.to_parquet(os.path.join(path, 'laion', 'analysis', f'sae_depth_{depth}_decoder_data_filtered.parquet'), index=False)
    print(len(filtered_df))

Most common top_indices patterns:
Pattern occurs 62675 times: (np.int64(5), np.int64(9), np.int64(2), np.int64(6), np.int64(8))... (truncated)
Pattern occurs 9 times: (np.int64(2006), np.int64(5), np.int64(2), np.int64(6), np.int64(9))... (truncated)
Pattern occurs 6 times: (np.int64(9160), np.int64(5), np.int64(2), np.int64(6), np.int64(9))... (truncated)
Pattern occurs 4 times: (np.int64(4355), np.int64(5), np.int64(2), np.int64(6), np.int64(9))... (truncated)
Pattern occurs 4 times: (np.int64(1304), np.int64(5), np.int64(2), np.int64(6), np.int64(9))... (truncated)
Pattern occurs 4 times: (np.int64(5891), np.int64(5), np.int64(2), np.int64(6), np.int64(9))... (truncated)
Pattern occurs 4 times: (np.int64(2789), np.int64(5), np.int64(2), np.int64(6), np.int64(9))... (truncated)
Pattern occurs 4 times: (np.int64(9600), np.int64(5), np.int64(2), np.int64(6), np.int64(9))... (truncated)
Pattern occurs 4 times: (np.int64(7044), np.int64(5), np.int64(2), np.int64(6), np.int64(9))... (trun